In [1]:
server = '140.114.77.15'
database = 'ChatService_EmotionTest'
username = 'sa'
password = '1Qazwsxedc'
driver= 'FreeTDS'

In [72]:
import pymssql
import codecs
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
import numpy as np
from sklearn import metrics

conn = pymssql.connect(server=server, user=username, password=password, database=database)

In [3]:
cursor = conn.cursor()  
cursor.execute('SELECT @@version;')  
row = cursor.fetchone() 
print(row)

('Microsoft SQL Server vNext (CTP1.3) - 14.0.304.138 (X64) \n\tFeb 13 2017 16:49:12 \n\tCopyright (C) 2016 Microsoft Corporation. All rights reserved.\n\ton Linux (Ubuntu 16.04.2 LTS)',)


In [4]:
cursor = conn.cursor()  
cursor.execute("SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE='BASE TABLE'")  
row = cursor.fetchone() 
print(row)
while row:  
    print(str(row[0]) + " " + str(row[1]) + " " + str(row[2]))
    row = cursor.fetchone()

('ChatService_EmotionTest', 'dbo', 'ChatList', 'BASE TABLE')
ChatService_EmotionTest dbo ChatList
ChatService_EmotionTest dbo ChatMessages
ChatService_EmotionTest dbo sysdiagrams


In [11]:
cursor.description

(('ChatId', 1, None, None, None, None, None),
 ('MessageTime', 4, None, None, None, None, None),
 ('SenderId', 1, None, None, None, None, None),
 ('MessageType', 1, None, None, None, None, None),
 ('Content', 1, None, None, None, None, None),
 ('MsgId', 3, None, None, None, None, None))

In [9]:
cursor = conn.cursor()  
cursor.execute("select top 30* from ChatMessages")  
row = cursor.fetchone() 
while row:  
    print(row)
    row = cursor.fetchone()

('2361157b-cc70-4bdb-98f6-d17cd7ba9d33', datetime.datetime(2015, 8, 26, 21, 51, 44, 247000), 'D425218C-3E8D-490B-84E9-737E9C58576C', 'text', '還在嗎？', 1)
('236309e1-2b95-4ab5-906e-929e5c1f30d8', datetime.datetime(2015, 7, 2, 8, 7, 46, 890000), 'EEC5CFA6-5B32-4281-82D1-3FA06C32CA84', 'text', '你好,padfone s更新5.0後,安裝在SD卡的程式都不能使用', 2)
('236309e1-2b95-4ab5-906e-929e5c1f30d8', datetime.datetime(2015, 7, 2, 8, 8, 33, 177000), 'A0110', 'text', '這部份,請您先把這些app移到手機內', 3)
('236309e1-2b95-4ab5-906e-929e5c1f30d8', datetime.datetime(2015, 7, 2, 8, 12, 19, 753000), 'A0110', 'text', '那請您在這邊找到後,移重到手機後,再移除', 4)
('2363f532-23b6-484a-b3cc-a7a7424822ce', datetime.datetime(2015, 9, 7, 8, 25, 34, 353000), 'CE88B5A2-E432-4B88-9DEE-D4CCB9C28484', 'text', '今年五月底購機,之前偶爾會重開機,近日手機不停發生自動重開機,今天光早上八點左右到八點半,已自動重開機三次,非常討厭,請盡速回覆處理。台北王律師', 5)
('22fbb230-9305-4c21-98d8-e7d2877c0d76', datetime.datetime(2016, 9, 29, 9, 48, 59, 263000), 's6262741s84s@gmail.com', 'text', '謝謝', 6)
('22fbc764-a52f-4317-a629-16c7ef51e5be', datetime.

In [6]:
cursor.execute('SELECT top 30* FROM ChatList where Survey is not null')

for row in cursor:
    print('row = %r' % (row,))
cursor.description

row = ('1c8fa7f5-0797-4b09-be5a-bbe48d212e69', 'TW', 'mobile', 'MOBILE', datetime.datetime(2015, 8, 19, 20, 51, 31, 857000), datetime.datetime(2015, 8, 19, 21, 15, 31, 227000), 0, 781, '1', datetime.datetime(2015, 8, 19, 21, 4, 37), None)
row = ('19c07cd6-e059-49ef-94fd-64a46ec10d23', 'TW', 'mobile', 'MOBILE', datetime.datetime(2015, 7, 25, 18, 41, 44, 267000), datetime.datetime(2015, 7, 25, 18, 54, 26, 227000), 0, 759, '2', datetime.datetime(2015, 7, 25, 18, 47, 38, 170000), None)
row = ('19c1466b-10ff-489a-8f60-34c4a1c50a52', 'TW', 'mobile', 'MOBILE', datetime.datetime(2015, 4, 25, 14, 16, 16, 210000), datetime.datetime(2015, 4, 25, 14, 36, 52, 687000), 0, 1233, '2', datetime.datetime(2015, 4, 25, 14, 36, 46, 943000), None)
row = ('19eadf8f-46ae-4d10-ab6c-c8e0e72395a3', 'TW', 'mobile', 'MOBILE', datetime.datetime(2015, 7, 27, 13, 53, 40, 257000), datetime.datetime(2015, 7, 27, 14, 1, 13, 423000), 0, 376, '1', datetime.datetime(2015, 7, 27, 14, 0, 10, 83000), None)
row = ('17ac9c34-74

(('ChatId', 1, None, None, None, None, None),
 ('TenantId', 1, None, None, None, None, None),
 ('Channel', 1, None, None, None, None, None),
 ('Skill', 1, None, None, None, None, None),
 ('CreateTime', 4, None, None, None, None, None),
 ('DisposeTime', 4, None, None, None, None, None),
 ('QueueMS', 3, None, None, None, None, None),
 ('Duration', 3, None, None, None, None, None),
 ('Survey', 1, None, None, None, None, None),
 ('SurveyTime', 4, None, None, None, None, None),
 ('Suggestion', 1, None, None, None, None, None))

In [12]:
cursor.execute('SELECT count(1) FROM ChatList')
cursor.fetchone() 

cursor.execute('SELECT count(1) FROM ChatList WHERE Survey = 1')
cursor.fetchone() 

# import codecs
# cursor.execute('SELECT Content FROM ChatList \
#                JOIN ChatMessages ON ChatList.ChatId = ChatMessages.ChatId \
#                WHERE ChatList.Survey IS NOT NULL AND \
#                LEN(ChatMessages.SenderId) > 5 AND\
#                ChatMessages.MessageType = \'text\'\
#                ORDER BY ChatMessages.ChatId\
#                ')
# f = codecs.open("./test_data/msg.txt","w", "utf-8-sig")
# for msg in cursor.fetchall():
#     f.write(msg[0]+'\n')

(44992,)

In [3]:
cursor = conn.cursor()  
cursor.execute("SELECT ChatList.ChatId, ChatMessages.SenderId , ChatMessages.Content from ChatList \
                JOIN ChatMessages ON ChatList.ChatId = ChatMessages.ChatId \
                WHERE ChatList.Survey = 1 AND ChatMessages.MessageType = \'text\' \
                ORDER BY ChatMessages.ChatId")  
row = cursor.fetchone() 
pos_data_sets = {}
while row:
    if row[0] in pos_data_sets:
        pos_data_sets[row[0]].append(row)
    else:
        pos_data_sets[row[0]] = [row]
    row = cursor.fetchone()

for key in list(pos_data_sets.keys()):
    user_content_array = []
    for data in pos_data_sets[key]:
        if data[1] == None or len(data[1]) > 5:
            user_content_array.append(data[2])
    pos_data_sets[key] = user_content_array

for key in list(pos_data_sets.keys()):
    if len(pos_data_sets[key]) == 0:
        del pos_data_sets[key]


In [4]:
cursor = conn.cursor()  
cursor.execute("SELECT ChatList.ChatId, ChatMessages.SenderId , ChatMessages.Content from ChatList \
                JOIN ChatMessages ON ChatList.ChatId = ChatMessages.ChatId \
                WHERE ChatList.Survey = 2 AND ChatMessages.MessageType = \'text\' \
                ORDER BY ChatMessages.ChatId")  
row = cursor.fetchone() 
neg_data_sets = {}
while row:
    if row[0] in neg_data_sets:
        neg_data_sets[row[0]].append(row)
    else:
        neg_data_sets[row[0]] = [row]
    row = cursor.fetchone()

for key in list(neg_data_sets.keys()):
    user_content_array = []
    for data in neg_data_sets[key]:
        if data[1] == None or len(data[1]) > 5:
            user_content_array.append(data[2])
    neg_data_sets[key] = user_content_array

for key in list(neg_data_sets.keys()):
    if len(neg_data_sets[key]) == 0:
        del neg_data_sets[key]

In [91]:
for index, row in enumerate(pos_data_sets):
    if index>20:
        break
    print(pos_data_sets[row])

['不知怎麼設定', 'ZF2', '滿意,謝謝你', '才能收到', '就是MMS沒有圖片', '文字收到', '還是要,放在卡1', '原來如此', '放到sim1', '之前用S3,可以收到mms', '應該不是中華電信的問題', '才能收到']
['那一個小時預約快修 的部分是?', '這樣要怎麼處理呢', '所以就有機會 當日維修好囉', '及最下面三個 案件  也沒反應', 'ZF2', '一個小時快修的部分', ' 恩恩  對', '不是', '如果送修的話 有可能當天就好嗎', '請問螢幕觸控按了沒反應', '沒使用單手模式下   任何情況那三個按鍵都沒反應', '還有螢幕中間部分區塊按了也沒反應', '恩   好   我了解了', '謝謝你', '550ML', '現在我只能轉換成單手操作變成小螢幕的虛擬三個觸控按鈕才能使用那三個案件的功能', '是因為沒辦法按 我才轉換成單手模式的']
['維修地點是板橋皇家沒錯', '瞭解, 感謝您~~~', '您好,請問官網上的維修進度查詢 如果顯示可以去取件了 是否就表示維修完成了呢？', '因為之前都會有簡訊通知 但我一直沒收到 剛剛線上查詢了 發現昨天就可以取件了', '江選帆', 'TWA3610915']
['ZENFONE 2', '你好 可以線上查詢保固日期嗎', 'F5AZFG21A099', '好的 謝謝']
['已傳輸到sd卡了', '檔案要放到那裡?', '我先試試,有問題再來請問您們', 'tks', '如已有解壓會有什麼狀況嗎？', '請問要如何手動更新下載軔體?', '我的手機是 ZenFone6', 'sd卡可以?', '這裡對嗎？', '已解壓縮檔要移除嗎？', 'ok ']
['你好', '我要把目前使用的手機號碼轉到另外一支,出門不用帶二支', '簡單明瞭', '我要將手機轉接,收到錯誤訊息,不能轉接出去', '簡單說我要將我的手機號碼轉至另一支', '我有開網路', 'zf3', '把我的電話轉到另外一支', '二支手機', '但不行', '謝了,了解', '不能轉接到另一支手機', '來電轉接無法轉接', '886', '網路傳回非預期回應', '通話設定==>來電轉接==>一律轉接==>輸入號碼後按開啟,出現網路傳回非預期回應']
['謝謝您', '請問一下', 'z

In [18]:
corpus = []
satisfied_degree = []
for key in pos_data_sets:
    corpus.append(' '.join(jieba.cut(' '.join(pos_data_sets[key]), cut_all=False)))
    satisfied_degree.append(1)

for key in neg_data_sets:
    corpus.append(' '.join(jieba.cut(' '.join(neg_data_sets[key]), cut_all=False)))
    satisfied_degree.append(-1)

In [34]:
corpus[:10]

['是 Z00AD   我 測試 ZF2     如果 將 朋友 的 簡訊移 到 " 私密 簡訊 " , 在 " 私密 簡訊 " 的 頁 面上 發出 簡訊給 設定 的 這個 電話 , 對方 收到 訊息 的 及 時 通知 回條 卻 會 顯示 其他人 的 電話 已 收到   ok     好 的 ! 謝謝 ! 拜拜 !   好 的 !   滿意   故意 試 了 更換 其他人 電話 一樣 有 此 情形   並不影響 什麼 , 只是 跟 您 說 一下 ZF2 有 這個 問題 而已   怎麼 查型號 ?     只 知道 是 2G , 32G 的   好   ZF2 很 好 用 ! 目前 很 健康 !     有 時會 要 更新 系統 , 一定 要 跟 著 更新 嗎 ？',
 '從 瀏覽器 下載 rar 檔案 都 一直 跑 出下載 失敗   zenfone5',
 '華碩 測試',
 '  我 的 手機   修理 回來 之 後   耗電 很快   請問 我 要 如何 知道 是 電池 的 問題 或者 是 其他 問題   好   有   已經 關閉   是',
 '莊佳容 , E4ATCY119602 , like790311 @ yahoo . com . tw   那麻煩 您 了 ！ 感謝   嗯   電量 到 50 % 左右 時會 自動 關機   0934360501   收到 簡訊 了 , 謝謝 您   我要 預約後天   請問 可以 預約 2 個 時段 嗎 ？   那 請 您 幫 我 約 14 ~ 15   以及 手機 平放 後 直立 , 喇叭 會 沒 有 聲音   嗯   請問 一下 , 時間 選擇 有 哪些 ？   再見   您好 , 可否 請 您 替 我 預約 台南 中西 區 皇家 俱樂部   謝謝 您 , 晚安',
 '謝謝 回覆   好 的 我 試試 看 , 謝謝 你   謝謝 您 , 再見 ^ ^   那 不好意思 , 我 還有 另個 問題 就是 , 我 8 月 28 才換 手機 , 插入 sim 卡後已 經有 兩次 顯示 沒有 插入 sim 卡 , 重新 開機 沒 多久 又 顯示 沒 插卡 , 都 要 把 背殼 拆開 重新 插入 sim 卡才 可以 , 這是 什 麽 原因 呢 ？ 謝謝   好 的 謝謝 您 的 回覆 , 希望 更新 後 一切 良好 , 

## 1.先將正面與負面的 使用者字詞 利用TFIDF 算出Bag of word
## 2.再將正負面丟去Train Classier 
## 3.在跟助教給的情緒LIB 比對分數看看



In [22]:
import plotly.graph_objs as go
from plotly.graph_objs import *
import plotly.plotly as py
import plotly.figure_factory as ff
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.model_selection import train_test_split

In [8]:
def generate_bar_chart(input_list, chart_title):
    x_axis = []
    y_axis = []
    for item in input_list:
        x_axis.append(item[0])
        y_axis.append(item[1])
    data = [go.Bar(
                x = x_axis,
                y = y_axis
        )]

    layout = go.Layout(
        title = chart_title,
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig, filename='basic-bar')

In [9]:
def display_scores(vectorizer, tfidf_result):
    # http://stackoverflow.com/questions/16078015/
    scores = zip(vectorizer.get_feature_names(),
                 np.asarray(tfidf_result.sum(axis=0)).ravel())
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    index_scores = []
    for index, item in enumerate(sorted_scores):
        index_scores.append((index, item[1]))
        
    generate_bar_chart(index_scores, 'TFIDF Score')
    return sorted_scores

In [20]:
# Utilizing TFIDF to delete useless words.
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(corpus)
tfidf_features = tfidf_vectorizer.get_feature_names()

In [37]:
sorted_scores = display_scores(tfidf_vectorizer, tfidf)


/home/phejimlin/anaconda3/envs/tensorflow_3.5/lib/python3.5/site-packages/plotly/plotly/plotly.py:218: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~phejimlin/0 or inside your plot.ly account where it is named 'basic-bar'


/home/phejimlin/anaconda3/envs/tensorflow_3.5/lib/python3.5/site-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Too Long



In [56]:
np_corpus = np.array(corpus)
np_satisfied_degree = np.array(satisfied_degree)

In [57]:
X_train, X_test, y_train, y_test = train_test_split(
    np_corpus, np_satisfied_degree, test_size=0.3, random_state=0)

In [73]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(X_train)
test_data_features = vectorizer.fit_transform(X_test)
# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()
test_data_features = test_data_features.toarray()

Creating the bag of words...



In [59]:
vocab = vectorizer.get_feature_names()
# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print(count, tag)

370 00
16 0010000300001
53 01
77 02
40 03
32 033559699
56 04
29 044000307
38 05
67 06
53 07
34 08
22 080
121 0800
15 0800093456
55 09
22 0k
886 10
271 100
33 1000
27 10050
79 100g
15 100gb
23 101
17 103
29 104
20 105
15 108
1425 11
24 111
593 12
22 123
21 12345
29 125
14 126
44 128g
259 13
23 1300
41 134
170 14
18 144
358 15
15 1500
19 156
17 1582
198 16
16 1600
15 165
185 16g
26 16gb
242 17
18 1700
228 18
43 1800
25 184
174 19
32 1900
16 194
25 196
31 198
20 1a
54 1g
445 20
29 200
78 2014
50 201407220024
171 2015
18 201501060109
15 20150117
26 20150527
31 20151211
116 2016
19 2017
235 21
355 22
15 220
210 23
324 24
104 25
27 256g
118 26
29 2600
80 27
88 28
96 29
61 2a
16 2ca
22 2f
532 2g
25 2gb
408 30
159 300
16 3000
67 31
30 317
80 32
460 32g
43 32gb
25 33
32 34
47 35
31 36
32 360
23 37
39 38
45 39
48 3c
22 3d
46 3d1
715 3g
14 3gb
385 3q
984 40
32 41
69 42
31 43
100 44
25 442
56 45
21 456
22 46
50 47
21 48
31 49
1600 4g
34 4g32g
18 4g64g
30 4gb
181 50
70 500
34 500cl
31 500kl
24 501


31 幾年
18 幾日
21 幾時
16 幾月
368 幾次
51 幾秒
31 幾種
91 幾號
313 幾點
1761 序號
64 底下
14 底部
52 店員
175 店家
116 店面
49 座充
16 庫裡
40 廠商
94 廠牌
23 廠過
21 廢話
87 廣告
39 廣播
26 延伸
70 延遲
57 延長
81 建國
34 建國店
52 建國路
50 建立
742 建議
18 建音樂
16 开机
19 弄掉
18 引導
47 張卡
66 張圖
15 張圖片
281 強制
20 強調
39 強迫
18 彈出
31 形鎖
39 彩盒
172 彰化
33 彰化市
76 影像
21 影幕
847 影片
60 影音
520 影響
22 往下拉
22 待料
27 待會
149 待機
62 很久
147 很困
1011 很多
190 很大
206 很小
64 很少
27 很差
45 很常
248 很快
238 很慢
39 很棒
47 很煩
25 很熱
46 很爛
17 很當
15 很耗
53 很長
47 很難
84 很麻煩
385 得到
42 得手
59 得知
20 從來
21 從手
19 從頭
26 循環
112 微信
19 心情
44 必要
22 必需
296 必須
191 忘記
46 忠孝
14 快下班
127 快修
362 快充
222 快取
91 快捷
31 快樂
28 快要
487 快速
16 快過
56 快遞
25 快門
96 快點
22 忽大忽小
76 忽然
31 怎么
16 怎摸
49 怎會
21 怎样
515 怎樣
45 怎用
19 怎看
30 怎處理
17 怎解
118 怎辦
7217 怎麼
17 怎麼換
36 怎麼查
129 怎麼樣
25 怎麼給
28 怎麼關
24 性能
29 怪怪
238 怪怪的
61 恩好
516 恩恩
27 恩恩好
20 恩謝謝
48 悠遊卡
2966 您好
19 您給
447 情形
772 情況
15 惡意
15 想像
19 想先
93 想到
14 想升級
362 想問
35 想想
65 想換
16 想查詢
17 想用
14 想維修
756 想要
50 想詢
57 愉快
20 意外
15 意度
1389 意思
17 意義
69 意見
29 感到
19 感受
622 感恩
170 感應
46 感應器
24 感激
41

17 發表
19 發覺
40 發送
45 白天
16 白痴
131 白色
25 白跑一趟
110 的關
30 的電
2950 皇家
843 皮套
479 盒子
169 盡快
16 盡速
46 盡量
3014 目前
20 目的
79 目錄
45 直到
2932 直接
44 直播
32 直營
29 直營店
23 直營門市
72 相信
277 相同
65 相容
51 相容性
1058 相機
15 相機會
281 相片
23 相當
174 相簿
440 相關
272 省電
35 看不出
274 看不到
117 看不懂
81 看來
14 看出
17 看別
1809 看到
47 看吧
20 看型號
78 看好
20 看官
22 看序號
23 看手
24 看清楚
2035 看看
30 看網
33 看能
32 看見
62 看起
157 看過
20 看錯
37 真人
14 真假
115 真是
14 真是太
30 真正
1039 真的
14 眼睛
20 睡前
54 睡覺
30 瞬間
411 瞭解
16 知到
4404 知道
19 砍掉
15 研發
102 研究
26 破掉
21 破損
23 破聲
264 破裂
107 破音
161 硬碟
361 硬體
45 碎裂
64 碰到
80 確保
46 確切
866 確定
58 確實
865 確認
20 碼是
23 碼牌
16 磨損
42 祝您
16 神器
23 神奇
30 神拍機
249 神腦
15 神魔
30 神麼
43 禁止
16 福利
62 福利品
448 禮拜
16 私人
18 科技
137 移到
370 移動
59 移至
40 移轉
511 移除
97 程序
29 程度
3624 程式
72 稍候片刻
59 稍微
30 稍早
79 稍等一下
231 穩定
48 空機
98 空白
857 空間
16 突出
1048 突然
28 窗上
34 窗口
46 立刻
176 立即
88 站上
104 竟然
30 端的
19 竹北
17 竹南
91 符合
194 符號
288 第一
49 第一代
25 第一天
16 第一支
360 第一次
133 第三
184 第三方
60 第三次
242 第二
26 第二代
18 第二天
96 第二季
26 第二支
120 第二次
19 第四
16 筆記本
152 筆電
53 等候
157 等到
166 等待
9

In [71]:
print("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features,  y_train)

Training the random forest...


In [74]:
predicted = forest.predict(test_data_features)

In [89]:
print(metrics.classification_report(y_test, predicted, target_names=['0', '1']))

             precision    recall  f1-score   support

          0       0.18      0.79      0.29      2445
          1       0.86      0.27      0.41     12184

avg / total       0.75      0.36      0.39     14629



In [38]:
#Bag of words to select top n = 400
bag_of_words = [word[0] for word in sorted_scores[:401]]

In [39]:
training_set = []
for sentence in corpus:
    features = []
    for word in sentence.split(' '):
        if word in bag_of_words:
            features.append(1)
        else:
            features.append(0)
    training_set.append(features)

In [40]:
X = np.array(training_set)
Y = np.array(satisfied_degree)
np.save('X_training.npy', X)
np.save('Y_training.npy', Y)

In [41]:
# df = pd.read_csv('./emotiondict/ntusd/ntusd40k.csv')
# display(df.head())

# X = df.drop('Polarity', 1)
# y = df['Polarity']
#X, y = df.iloc[:, 1:].values, df.iloc[:, 0].values

# split X into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.3, random_state=0)

print('#Training data points: %d' % X_train.shape[0])
print('#Testing data points: %d' % X_test.shape[0])
print('Class labels:', np.unique(Y))

#Training data points: 34133
#Testing data points: 14629
Class labels: [-1  1]


In [42]:
print(X_train[:5])
print(y_train[:5])

[ [0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]
 [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,

In [90]:
from sklearn.tree import DecisionTreeClassifier

# criterion : impurity function
# max_depth : maximum depth of tree
# random_state : seed of random number generator
tree = DecisionTreeClassifier(criterion='entropy', 
                              max_depth=3, 
                              random_state=0)
tree.fit(train_data_features,  y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=0, splitter='best')

In [94]:
y_pred = tree.predict(test_data_features)

# a more convenient way to evaluate a trained model is to use the sklearn.metrics 
from sklearn.metrics import accuracy_score
print('Accuracy (tree, sklearn): %.2f' % accuracy_score(y_test, y_pred))
# print(metrics.classification_report(y_test, y_pred, target_names=['0', '1']))

Accuracy (tree, sklearn): 0.83
